In [1]:
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
from dask import delayed
from dask.distributed import Client
from dask_sql import Context

Question 1

In [2]:
rests = dd.read_csv('C:/Users/poona/Downloads/Restaurants_in_Durham_County_NC.csv',sep=';',dtype={'Closing_Date': 'object',
       'Hours_Of_Operation': 'object',
       'Premise_Zip': 'object'})

In [3]:
F1=rests[rests.Status == 'ACTIVE']  

In [4]:
F2=F1[F1.Rpt_Area_Desc == 'Food Service']  

In [5]:
c = Context()
c.create_table("F2", F2)

In [6]:
result=c.sql("""
    SELECT
        "Premise_Name", "geolocation"
    FROM F2
""")

In [7]:
F3 = result.compute()

In [8]:
F3=F3.dropna(subset=['geolocation'])  

In [9]:
def lat(df):
    dt=df.geolocation
    return float(dt.split()[0][:-1])

In [10]:
F3['lat']=F3.apply(lat,axis=1) #https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.apply.html

In [11]:
def long(df):
    dt=df.geolocation
    return float(dt.split()[1])

In [12]:
F3['long']=F3.apply(long,axis=1)

In [13]:
F3 #I noticed that dask switches between using a dask dataframe and pandas dataframe, I put this here because I do bring in pandas next cell only to open a file and didn't notice a difference other than viewing appearance between the 2 types of dataframes and how they interact with the functions I used in this assignment

,Premise_Name,geolocation,lat,long
0,WEST 94TH ST PUB,"35.9207272, -78.9573299",35.920727,-78.957330
1,BROOKDALE DURHAM IFS,"36.0467802, -78.8895483",36.046780,-78.889548
2,SMOOTHIE KING,"35.9182655, -78.9593263",35.918265,-78.959326
3,HAMPTON INN & SUITES,"36.0183378, -78.9060312",36.018338,-78.906031
10,KROGER R 381 MEAT MARKET,"35.9495321, -78.9211694",35.949532,-78.921169
...,...,...,...,...
2453,LA QUINTA INN AND SUITES,"35.9454822, -78.9724505",35.945482,-78.972450
2454,NINTH STREET BAKERY,"35.9968356, -78.9023636",35.996836,-78.902364
2455,BRUEGGERS BAGELS,"36.0075308, -78.9219898",36.007531,-78.921990
2456,WIMPY'S GRILL,"36.013017, -78.9321444",36.013017,-78.932144


In [14]:
import pandas as pd
dummy = pd.read_json('C:/Users/poona/Downloads/durham-nc-foreclosure-2006-2016.json')

In [15]:
forclosures = dd.from_pandas(dummy,npartitions=1) #Converting to dask, just had pandas to deal with the square brackets in the file

In [16]:
c.create_table("forclosures", forclosures)

In [17]:
result=c.sql("""
    SELECT
        fields
    FROM forclosures
""")

In [18]:
geom = result.compute()

In [19]:
#Doesn't do anything, just have it here to rememeber the fucntion in the future
#geom = geom.astype({"fields": str}).set_index('fields')

In [20]:
def coord(df):
    dt=df.fields
    return dt.get('geocode')

In [21]:
geom['fields']=geom.apply(coord,axis=1)

In [22]:
geom=geom.dropna(subset=['fields'])  

In [23]:
def lat(df):
    dt=df.fields
    return float(dt[0])

In [24]:
geom['glat']=geom.apply(lat,axis=1)

In [25]:
def long(df):
    dt=df.fields
    return float(dt[1])

In [26]:
geom['glong']=geom.apply(long,axis=1)

In [27]:
CJ=F3.merge(geom, how='cross') #Got it from here https://docs.dask.org/en/stable/generated/dask.dataframe.multi.merge.html

In [28]:
from haversine import haversine, Unit
def distance(df):
    x=df.lat
    y=df.long
    z=df.glat
    w=df.glong
    return float(haversine((x,y),(z,w), unit=Unit.MILES))

In [29]:
CJ['distance']=CJ.apply(distance,axis=1)

In [30]:
c.create_table("CJ", CJ)

In [31]:
result=c.sql("""
    SELECT
        *
    FROM CJ
    WHERE distance<=1
""")

In [32]:
CJ1 = result.compute()

In [33]:
CJ1

,Premise_Name,geolocation,lat,long,fields,glat,glong,distance
263,WEST 94TH ST PUB,"35.9207272, -78.9573299",35.920727,-78.957330,"[35.9128174, -78.9433162]",35.912817,-78.943316,0.955817
497,WEST 94TH ST PUB,"35.9207272, -78.9573299",35.920727,-78.957330,"[35.915097, -78.941141]",35.915097,-78.941141,0.985859
521,WEST 94TH ST PUB,"35.9207272, -78.9573299",35.920727,-78.957330,"[35.9128174, -78.9433162]",35.912817,-78.943316,0.955817
568,WEST 94TH ST PUB,"35.9207272, -78.9573299",35.920727,-78.957330,"[35.915097, -78.941141]",35.915097,-78.941141,0.985859
1141,WEST 94TH ST PUB,"35.9207272, -78.9573299",35.920727,-78.957330,"[35.915097, -78.941141]",35.915097,-78.941141,0.985859
...,...,...,...,...,...,...,...,...
2112656,WIMPY'S GRILL,"36.013017, -78.9321444",36.013017,-78.932144,"[36.01864, -78.916274]",36.018640,-78.916274,0.968301
2112782,WIMPY'S GRILL,"36.013017, -78.9321444",36.013017,-78.932144,"[36.01864, -78.916274]",36.018640,-78.916274,0.968301
2112808,WIMPY'S GRILL,"36.013017, -78.9321444",36.013017,-78.932144,"[36.01864, -78.916274]",36.018640,-78.916274,0.968301
2112939,CILANTRO MEDITERREAN GRILL & PIZZA,"35.8822361, -78.8511586",35.882236,-78.851159,"[35.8912585, -78.845683]",35.891258,-78.845683,0.694668


Question 2

In [34]:
b = db.read_text('C:/Users/poona/Downloads/hw1dir/hw1/*.txt')  

In [35]:
df = b.to_dataframe()

In [36]:
import re
def punct(df):
    dt=df[0]
    return re.sub('[^A-Za-z0-9]+', ' ', dt)

In [37]:
df['punct']=df.apply(punct,axis=1)

C:\Users\poona\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\core.py:5912: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


In [38]:
def lower(df):
    dt=df.punct
    return dt.lower()

In [39]:
df['lower']=df.apply(lower,axis=1)

C:\Users\poona\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\core.py:5912: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


In [40]:
def less(df):
    dt=df.lower
    return len(dt)

In [41]:
df['less']=df.apply(less,axis=1)

C:\Users\poona\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\core.py:5912: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'int64'))

  warnings.warn(meta_warning(meta))


In [42]:
df1=df[df.less > 3]  

In [43]:
df2=df1[['lower']]

In [44]:
def split(df):
    dt=df.lower
    return dt.split()

In [45]:
df2['split']=df2.apply(split,axis=1)

C:\Users\poona\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\core.py:5912: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


In [46]:
df3=df2.explode('split')  

In [47]:
len(df3) #Total number of words

9578487

In [48]:
df3.head()

,lower,split
1,4000241 i think it is safe to say that ours i...,4000241
1,4000241 i think it is safe to say that ours i...,i
1,4000241 i think it is safe to say that ours i...,think
1,4000241 i think it is safe to say that ours i...,it
1,4000241 i think it is safe to say that ours i...,is


In [49]:
df4 = df3.groupby(['split']).count()

In [50]:
len(df4) #Total number of unique words

136090

In [51]:
df4.head()

,lower
split,
0,2252
00,560
000,3712
0000,10
00000,1


In [52]:
#To get specific value df4.loc['0','lower'].head().values[0]

Above was unigrams, working on bigrams now

In [53]:
df2.head()

,lower,split
1,4000241 i think it is safe to say that ours i...,"[4000241, i, think, it, is, safe, to, say, tha..."
2,4000341 the high point of my freshman year en...,"[4000341, the, high, point, of, my, freshman, ..."
3,4000441 there seems only one cause behind all...,"[4000441, there, seems, only, one, cause, behi..."
4,4000541 the papers that comprise this symposi...,"[4000541, the, papers, that, comprise, this, s..."
5,4000641 quaesivit arcana poli videt dei he so...,"[4000641, quaesivit, arcana, poli, videt, dei,..."


In [54]:
def split2(df):
    dt=df.split
    dt1=dt[1:]
    dt2=dt[:-1]
    records = []
    for i in range(len(dt2)):
        records.append( dt2[i] + ' ' + dt1[i])
    return records

In [55]:
bdf2 = df2

In [56]:
bdf2['split2']=df2.apply(split2,axis=1)

C:\Users\poona\AppData\Roaming\Python\Python310\site-packages\dask\dataframe\core.py:5912: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


In [57]:
bdf3=bdf2.explode('split2')  

In [58]:
bdf3.head()

,lower,split,split2
1,4000241 i think it is safe to say that ours i...,"[4000241, i, think, it, is, safe, to, say, tha...",4000241 i
1,4000241 i think it is safe to say that ours i...,"[4000241, i, think, it, is, safe, to, say, tha...",i think
1,4000241 i think it is safe to say that ours i...,"[4000241, i, think, it, is, safe, to, say, tha...",think it
1,4000241 i think it is safe to say that ours i...,"[4000241, i, think, it, is, safe, to, say, tha...",it is
1,4000241 i think it is safe to say that ours i...,"[4000241, i, think, it, is, safe, to, say, tha...",is safe


In [59]:
len(bdf3) #Total number of bigrams

9573750

In [60]:
bdf4 = bdf3.groupby(['split2']).count()

In [61]:
len(bdf4) #Total number of unique bigrams

2484908

In [62]:
bdf4.head()

,lower,split
split2,,
0 0,147,147
0 00,7,7
0 000,8,8
0 00000,1,1
0 00001,1,1


In [63]:
#To get specific value bdf4.loc['0 0','lower'].head().values[0]

Approximation

In [64]:
#Can just repalce the word1 and word2 variables with what you want to find the approximation

In [65]:
word1='0'
word2='0'
combined = word1 + ' ' + word2
x = df4.loc[word1,'lower'].head().values[0]/len(df3)
y = bdf4.loc[combined,'lower'].head().values[0]/len(bdf3)
y/x

0.06530760843475339